In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np
from pyod.models.iforest import IForest  # Modelo Isolation Forest para detección de anomalías

# Configuración de conexión a la base de datos
DB_CONFIG = {
    "user": "root",
    "password": "1234",
    "host": "127.0.0.1",
    "port": "3306",
    "database": "gp-mlp-telemtry"
}

# Crear la cadena de conexión
DATABASE_URL = f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
engine = create_engine(DATABASE_URL)

# Obtener todas las estaciones
def obtener_estaciones():
    with engine.connect() as conn:
        result = conn.execute(text("CALL `gp-mlp-telemtry`.GetEstaciones()"))
        datos = result.fetchall()
        columnas = result.keys()
    return pd.DataFrame(datos, columns=columnas)[["estacion_id", "nombre"]]

# Obtener todos los sensores de una estación
def obtener_sensores(estacion_id):
    with engine.connect() as conn:
        result = conn.execute(text("CALL `gp-mlp-telemtry`.GetSensoresByEstacion(:estacion)"), {"estacion": estacion_id})
        datos = result.fetchall()
        columnas = result.keys()
    return pd.DataFrame(datos, columns=columnas)[["sensor_id", "estacion_id", "tipo"]]

# Obtener datos de un sensor específico
def obtener_datos(estacion_id, sensor_id):
    with engine.connect() as conn:
        result = conn.execute(text("CALL `gp-mlp-telemtry`.GetSensorData(:estacion, :sensor, :periodo)"), {
            "estacion": estacion_id,
            "sensor": sensor_id,
            "periodo": 3  # Últimos datos disponibles
        })
        datos = result.fetchall()
        columnas = result.keys()
    df = pd.DataFrame(datos, columns=columnas)[["fecha_hora", "valor"]]
    
    # Manejo de valores NaN en la columna 'valor'
    df.dropna(subset=["valor"], inplace=True)  # Eliminar filas con NaN en 'valor'
    
    return df


# Función para detectar outliers según límites definidos en la base de datos
def detectar_outliers(estacion_id, sensor_id, datos):
    # Consultar los límites del sensor desde la base de datos
    with engine.connect() as conn:
        result = conn.execute(text("""
            SELECT limite_inferior, limite_superior 
            FROM `gp-mlp-telemtry`.Sensores 
            WHERE sensor_id = :sensor_id AND estacion_id = :estacion_id
        """), {"sensor_id": sensor_id, "estacion_id": estacion_id})
        limites = result.fetchone()
    
    if not limites or limites["limite_inferior"] is None or limites["limite_superior"] is None:
        print(f"[Advertencia] Sensor {sensor_id} no tiene límites definidos.")
        return pd.DataFrame()
    
    limite_inf = limites["limite_inferior"]
    limite_sup = limites["limite_superior"]
    
    # Detectar valores fuera de los límites
    datos_outliers = datos[(datos["valor"] < limite_inf) | (datos["valor"] > limite_sup)]

    for _, fila in datos_outliers.iterrows():
        print(f"[Outlier] Sensor: {sensor_id}, Estación: {estacion_id}, Fecha: {fila['fecha_hora']}, Valor: {fila['valor']}, Límites: [{limite_inf}, {limite_sup}]")
    
    return datos_outliers
   
# Función para detectar anomalías con PyOD
def detectar_anomalias(datos):
    if len(datos) < 10:
        return pd.DataFrame()
    
    X = datos["valor"].values.reshape(-1, 1)
    
    # Verificar si aún hay valores NaN
    if np.isnan(X).any():
        return pd.DataFrame()
    
    modelo = IForest(contamination=0.1)
    modelo.fit(X)
    datos["anomaly"] = modelo.predict(X)
    return datos[datos["anomaly"] == 1]  # 1 indica anomalía en PyOD



In [2]:
if __name__ == "__main__":
    # Puedes elegir qué tipo de detección activar
    buscar_outliers = True
    buscar_anomalias = False

    estaciones = obtener_estaciones()
    sensores_totales = []

    for _, estacion in estaciones.iterrows():
        sensores = obtener_sensores(estacion["estacion_id"])
        sensores_totales.append(sensores)

    sensores_df = pd.concat(sensores_totales, ignore_index=True)

    print("Buscando puntos críticos...")

    for _, sensor in sensores_df.iterrows():
        datos = obtener_datos(sensor["estacion_id"], sensor["sensor_id"])
        if not datos.empty:
            
            if buscar_outliers:
                outliers = detectar_outliers(sensor["estacion_id"], sensor["sensor_id"], datos)

            if buscar_anomalias:
                anomalias = detectar_anomalias(datos)
                if not anomalias.empty:
                    for _, fila in anomalias.iterrows():
                        print(f"[Anomalia] Sensor: {sensor['sensor_id']}, Estación: {sensor['estacion_id']}, Tipo: {sensor['tipo']}, Fecha: {fila['fecha_hora']}, Valor: {fila['valor']}")


Buscando puntos críticos...


TypeError: tuple indices must be integers or slices, not str